In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
import json

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Embedding, SpatialDropout1D, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import io


In [2]:
PPRINT_WIDTH = 160 # константа для функции pprint, количество символов в одной строке при выводе

In [ ]:
path_train = '/kaggle/input/train-nlp/train.json'

with open(path_train, 'r') as f:
    data_train = json.loads(f.read())

text_data = pd.DataFrame(data_train)
text_data.drop(columns = 'id', inplace = True)
print(text_data.shape)
print(text_data.sample(5))

In [ ]:
n_classes = text_data['sentiment'].nunique()
n_classes

In [ ]:
#  сделаем разделение наших данных на обучение тест с учетом стратификации
train_index, test_index = train_test_split(np.arange(text_data.shape[0]), stratify = text_data['sentiment'])

x_train_raw = text_data.iloc[train_index, 0].values
y_train_raw = text_data.iloc[train_index, 1].values
x_test_raw = text_data.iloc[test_index, 0].values
y_test_raw = text_data.iloc[test_index, 1].values

In [ ]:
NUM_WORDS = 20000 # количество слов в словаре

# объявляем наш tokenizer
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                      lower=True, split=' ', char_level=False, oov_token='UNKNOWN'
                     )
# обучаем tokenizer на текстах, составляем словарь частотности
tokenizer.fit_on_texts(x_train_raw)

In [ ]:
# переводим наши тексты в последовательность индексов (токенов) с помощью tokenizer
x_train_seq = tokenizer.texts_to_sequences(x_train_raw)
x_test_seq = tokenizer.texts_to_sequences(x_test_raw)

In [ ]:
# объявим функцию для чистки наших последовательностей от тега unknown
# мы предполагаем, что наличие тега unknown не несет значимой информации
def drop_UNKNOWN (x_seq, unknown=1):
    x_seq_short = []
    for x in x_seq:
        x_ = np.array(x)
        x_ = x_[np.where(x_ !=unknown )]
        x_seq_short.append(list(x_))
    return x_seq_short

In [ ]:
# очистим наши последовательности, полученные из обучающей и тестовой выборок
# от тега unknown с использованием объявленной функции
x_train_seq_short = drop_UNKNOWN(x_train_seq)
x_test_seq_short = drop_UNKNOWN(x_test_seq)

In [ ]:
# устанавливаем максимальную длинну последовательности токенов
MAX_LEN_SEQ = 2000

In [ ]:
# вырвниваем длинну всех последовательностей токенов до MAX_LEN_SEQ
# при помощи стандартного инструмента pad_sequence, входящего в Keras
# при этом последовательности короче MAX_LEN_SEQ будут дополнены нулями
# а последовательности длиннее MAX_LEN_SEQ будут обрезаны

x_train_emb = pad_sequences(x_train_seq_short, padding='post', maxlen=MAX_LEN_SEQ)
x_test_emb = pad_sequences(x_test_seq_short, padding='post', maxlen=MAX_LEN_SEQ)

In [ ]:
# объявляем кодировщик для целевого признака (класса отзыва)
# используем стандартный OneHotEncoder из библиотеки Sklearn
target_encoeder = OneHotEncoder(sparse=False)

# обучаем наш кодировщик на целевых призаках обучающей выборки
target_encoeder.fit(y_train_raw.reshape([-1, 1]))

In [ ]:
# объявляем переменную classes_names, в которой сохраним матрицу
# с названиями классов (тип отзыва)
# названия классов понадобятся нам на этапе инференса
classes_names = target_encoeder.categories_[0]
classes_names

In [ ]:
# переведем целевые переменные для обучающей и тестовой выборки в формат OHE
# это нужно для подачи в модель

y_train_01 = target_encoeder.transform(y_train_raw.reshape([-1, 1]))
y_test_01 = target_encoeder.transform(y_test_raw.reshape([-1, 1]))

In [ ]:
modelEmb = Sequential() # объявляем нашу модель как последовательность слоев
# добавляем слой Embedding
modelEmb.add(Embedding(input_dim=NUM_WORDS, output_dim=200, input_length=MAX_LEN_SEQ))
# добавляем слой SpatialDropout1D для "прореживания" и борьбы с переобучением
modelEmb.add(SpatialDropout1D(0.5))

# Добавляем слой долго-краткосрочной памяти (400 элементов для долговременного хранения информации, отключаем входной сигнал с вероятностью 20%, отключаем рекуррентный сигнал с вероятностью 20%)
modelEmb.add(LSTM(400, dropout = 0.2, recurrent_dropout = 0.2))

# добавим выравнивающий слой
modelEmb.add(Flatten())
# добавим Dense слой на 16 нейронов
modelEmb.add(Dense(96,  activation='relu'))
# добавим Dense слой на 16 нейронов
modelEmb.add(Dense(16,  activation='relu'))
# добавим выходной полносвязный слой для классификации
modelEmb.add(Dense(n_classes,activation='softmax'))
modelEmb.add(Dropout(0.2))
# компилируем модель
modelEmb.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy',  metrics=['accuracy'])

# выводим данные по модели
modelEmb.summary()

In [ ]:
# обучаем модель
modelEmb.fit(x = x_train_emb,  y = y_train_01, epochs = 10, verbose = 1, 
             validation_data= (x_test_emb, y_test_01))